In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='http://publichealth.lacounty.gov/media/Coronavirus/locations.htm'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [5]:
def dataframe_from_tr(tr_elements, length):
    elements = []
    for el in tr_elements:
        if len(el) == length:
            elements.append(el)
            
    dfs = []
    
    col=[]
    i=0
    for t in elements[0]:
        i+=1
        name=t.text_content()
        col.append((name,[]))
        
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1, len(elements)):
        #T is our j'th row
        T=elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=length:
            break

        if 'blue text-white' in T.values():
            Dict={title:column for (title,column) in col}
            df=pd.DataFrame(Dict)
            dfs.append(df)
            i=0
            #For each row, store each first element (header) and an empty list
            for t in T.iterchildren():
                if i == 0:
                    name=t.text_content()
                    col[i] = (name,[])
                else:
                    col[i] = (col[i][0],[])
                i+=1

        else:
            #i is the index of our column
            i=0


            #Iterate through each element of the row
            for t in T.iterchildren():
                data=t.text_content() 
                #Check if row is empty
                if i>0:
                #Convert any numerical value to integers
                    try:
                        data=int(data)
                    except:
                        pass
                #Append the data to the empty list of the i'th column
                col[i][1].append(data)
                #Increment i for the next column
                i+=1
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    dfs.append(df)
    
    return dfs

In [17]:
dfs = dataframe_from_tr(tr_elements, 3)

In [18]:
dfs += dataframe_from_tr(tr_elements, 1)

In [19]:
def show(dfs, i):
    n = 5
    return dfs[i].head(n)

In [20]:
show(dfs, 0)

,Locations,Rate**,Total Cases
0,Laboratory Confirmed Cases (LCC),,6360
1,- Los Angeles County (excl. LB and Pas),,6089
2,- Long Beach,,213
3,- Pasadena,,58
4,,,


In [21]:
show(dfs, 1)

,Deaths,Rate**,Total Cases
0,- Los Angeles County (excl. LB and Pas),,142
1,- Long Beach,,3
2,- Pasadena,,2
3,,,
4,,,


In [22]:
show(dfs, 2)

,Age Group (Los Angeles County Cases Only-excl LB and Pas),Rate**,Total Cases
0,0 to 17,,59
1,18 to 40,,2072
2,41 to 65,,2650
3,Over 65,,1278
4,Under Investigation,,30


In [23]:
show(dfs, 3)

,Gender (Los Angeles County Cases Only-excl LB and Pas),Rate**,Total Cases
0,Male,,3082
1,Female,,2763
2,Under Investigation,,244
3,,,
4,,,


In [24]:
show(dfs, 4)

,Hospitalization LAC cases only (not Long Beach and Pasadena),Rate**,Total Cases
0,Hospitalized (Ever),,1366
1,,,
2,,,


In [25]:
show(dfs, 5)

,CITY/COMMUNITY**,Rate**,Total Cases
0,City of Agoura Hills,85.03,18
1,City of Alhambra,25.37,22
2,City of Arcadia,28.07,16
3,City of Artesia,--,--
4,City of Avalon,0,0


In [26]:
show(dfs, 6)

,Institutional Settings with a Single Confirmed Positive Case
0,Union Rescue Mission
1,LA County Jail – Twin Towers
2,"Worthams Supported Living, Inglewood"
3,"Milestones Ranch, Malibu"
4,"Canon Human Services, LA"
